In [1]:
import boto3
import botocore
import csv
from datetime import tzinfo, timedelta, datetime, date
import datetime
import glob
from email.message import EmailMessage
import os.path, time
import smtplib
import creds

In [2]:
# Which market is reported
market = 'Gainesville and Ocala'

In [6]:
BUCKET_NAME = 'cinemascrape' 

today = datetime.date.today()
scrapedate = str(today.strftime("%Y-%m-%d"))
KEY = "fandango/fandango-" + scrapedate + ".csv"

s3 = boto3.resource('s3')

try:
    s3.Bucket(BUCKET_NAME).download_file(KEY, 'scrapes/cinemascrape.csv')
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise

In [ ]:
# SEND REPORT to recipient(s).
#with open(bigreport) as fp:
# Create a text/plain message
msg = EmailMessage()
msg.set_content("No content yet")

sender = creds.gmail_address
receiver = creds.receiver
password = creds.gmail_password

msg['Subject'] = f'Latest cinema scrape for {market}'
msg['from'] = sender
msg['To'] = receiver

# Send the message via our own SMTP server.
try:
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login(sender, gmail_password)
    server.send_message(msg)
    server.quit()
except:
    print('Something went wrong...')